In [12]:
import pandas as pd

multivariate_df = pd.read_csv(
    "/media/maciek/207GB/Documents/PhD/git/metro3/notebooks/results/DL-summary-models-accuracy/DL-summary-multivariate.csv")
multivariate_df = multivariate_df.rename(columns={"score": "ModelAccuracy"})
del multivariate_df["status"]
multivariate_df["type"] = "multivariate"
multivariate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   dataset        30 non-null     object 
 1   ModelAccuracy  20 non-null     float64
 2   type           30 non-null     object 
dtypes: float64(1), object(2)
memory usage: 848.0+ bytes


In [9]:
univariate_df = pd.read_csv(
    "/media/maciek/207GB/Documents/PhD/git/metro3/notebooks/results/DL-summary-models-accuracy/DL-summary-univariate.csv")
univariate_df = univariate_df.rename(columns={"score": "ModelAccuracy"})
del univariate_df["status"]
univariate_df["type"] = "univariate"
univariate_df


,dataset,ModelAccuracy,type
0,DiatomSizeReduction,100.000000,univariate
1,DistalPhalanxOutlineAgeGroup,77.777779,univariate
2,DistalPhalanxOutlineCorrect,78.082192,univariate
3,DistalPhalanxTW,66.666669,univariate
4,DodgerLoopDay,55.000001,univariate
...,...,...,...
102,Wine,60.714287,univariate
103,WordSynonyms,67.841411,univariate
104,Worms,50.769234,univariate
105,WormsTwoClass,50.769234,univariate


In [13]:
data_records = []

# Podaj ścieżkę do pliku
file_path = "/media/maciek/207GB/Documents/PhD/git/metro3/scripts/report (copy).history"

# Wczytywanie pliku linia po linii
with open(file_path, 'r', encoding='utf-8') as file:
    for line in file:
        line = line.strip()
        if not line:
            continue  # pomijamy puste linie

        # Zakładamy, że linia ma strukturę:
        # <type_and_name>: <metric1>:<value1>, <metric2>:<value2>, <metric3>:<value3>
        # np. "multivariate_ArticularyWordRecognition: Anchor:Yes, Lime:Yes, Shap:Yes"
        try:
            header, metrics_part = line.split(": ", 1)
        except ValueError:
            # Pomiń linie niezgodne z oczekiwanym formatem
            continue

        # Rozdzielamy header na typ i nazwę modelu, używając separatora "_"
        if "_" in header:
            type_token, name_token = header.split("_", 1)
        else:
            # Jeśli brak separatora, traktujemy całość jako nazwę i ustawiamy typ jako 'unknown'
            type_token = 'unknown'
            name_token = header

        # Upewniamy się, że typ jest jednym z oczekiwanych: multivariate lub univariate
        type_value = type_token.lower()
        if type_value not in ["multivariate", "univariate"]:
            type_value = "unknown"

        # Przygotowujemy słownik dla bieżącej linii
        record = {
            "type": type_value,
            "name": name_token,
            "Anchor": None,
            "Lime": None,
            "Shap": None
        }

        # Rozdzielamy część metryk – zakładamy, że są rozdzielone przecinkiem i odstępem
        metric_pairs = metrics_part.split(", ")
        for metric in metric_pairs:
            # Każda metryka ma format Key:Value, więc dzielimy po pierwszym dwukropku
            try:
                key, value = metric.split(":", 1)
            except ValueError:
                continue  # pomijamy nieprawidłowe pary
            key = key.strip()
            value = value.strip()
            # Aktualizujemy słownik, tylko jeśli klucz jest jednym z oczekiwanych
            if key in record:
                record[key] = value

        data_records.append(record)

# Tworzymy DataFrame z listy słowników
df = pd.DataFrame(data_records)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   type    103 non-null    object
 1   name    103 non-null    object
 2   Anchor  103 non-null    object
 3   Lime    103 non-null    object
 4   Shap    103 non-null    object
dtypes: object(5)
memory usage: 4.1+ KB


In [16]:


# Zakładamy, że ramki multivariate_df, univariate_df oraz df są już załadowane.
# Przykładowo mogłyby być wczytane wcześniej z plików CSV lub poprzez inne metody.

# Dodajemy nową kolumnę do df, w której będziemy umieszczać znalezioną wartość accuracy.
df["ModelAccuracy"] = None

# Iterujemy po każdej linii w df przy użyciu iterrows
for index, row in df.iterrows():
    # Pobieramy typ modelu i nazwę (dataset) z danej linii
    model_type = row["type"].strip().lower()  # np. "multivariate" lub "univariate"
    model_name = row["name"].strip()
    accuracy_value = None

    # W zależności od typu, przeszukujemy odpowiednią ramkę:
    if model_type == "multivariate":
        subset = multivariate_df[multivariate_df["dataset"] == model_name]
        if not subset.empty:
            # Jeśli znaleziono więcej niż jeden wiersz – pobieramy pierwszy
            accuracy_value = subset.iloc[0]["ModelAccuracy"]
    elif model_type == "univariate":
        subset = univariate_df[univariate_df["dataset"] == model_name]
        if not subset.empty:
            accuracy_value = subset.iloc[0]["ModelAccuracy"]

    # Wpisujemy znalezioną wartość do nowej kolumny
    df.at[index, "ModelAccuracy"] = accuracy_value

# Wyświetlamy wynik końcowy
final_order = ["type", "name", "ModelAccuracy", "Anchor", "Lime", "Shap"]
df = df[final_order]
df


,type,name,ModelAccuracy,Anchor,Lime,Shap
0,multivariate,ArticularyWordRecognition,96.527779,Yes,Yes,Yes
1,multivariate,AtrialFibrillation,37.5,Yes,Yes,Yes
2,multivariate,BasicMotions,100.0,Yes,Yes,Yes
3,multivariate,Cricket,93.333334,Yes,Yes,Yes
4,multivariate,Epilepsy,91.30435,Yes,Yes,Yes
...,...,...,...,...,...,...
98,univariate,Wine,60.714287,No,Yes,Yes
99,univariate,WordSynonyms,67.841411,No,Yes,Yes
100,univariate,Worms,50.769234,No,Yes,Yes
101,univariate,WormsTwoClass,50.769234,Yes,Yes,Yes


In [17]:
csv_path = "/media/maciek/207GB/Documents/PhD/git/metro3/scripts/report.csv"
df.to_csv(csv_path, index=False)

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103 entries, 0 to 102
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   type           103 non-null    object
 1   name           103 non-null    object
 2   ModelAccuracy  90 non-null     object
 3   Anchor         103 non-null    object
 4   Lime           103 non-null    object
 5   Shap           103 non-null    object
dtypes: object(6)
memory usage: 5.0+ KB


In [33]:
def format_accuracy(x):
    try:
        # Próba konwersji do float i formatowanie do 2 miejsc po przecinku;
        # Następnie zamiana kropki na przecinek
        return f"{float(x):.2f}".replace('.', ',')
    except:
        return "TBD"

df_copy = df.copy()
# Uzupełnienie i przetworzenie kolumny ModelAccuracy
# Wykorzystujemy apply, żeby przekształcić dostępne wartości do formatu z 2 miejscami po przecinku
df_copy["Model Accuracy"] = df["ModelAccuracy"].apply(format_accuracy)

# Przemianowanie kolumn:
# - "name" -> "Time Series"
# - "Lime" i "Shap" na "LIME" oraz "SHAP"
df_copy = df_copy.rename(columns={"type": "type", "name": "Time Series", "Lime": "LIME", "Shap": "SHAP"})


df_copy.loc[df_copy["Time Series"] == "Adiac", "Model Accuracy"] = "29.59"
df_copy.loc[df_copy["Time Series"] == "Beef", "Model Accuracy"] = "53.33"
df_copy.loc[df_copy["Time Series"] == "BeetleFly", "Model Accuracy"] = "80.00"
df_copy.loc[df_copy["Time Series"] == "BirdChicken", "Model Accuracy"] = "50.00"
df_copy.loc[df_copy["Time Series"] == "BME", "Model Accuracy"] = "86.67"
df_copy.loc[df_copy["Time Series"] == "CBF", "Model Accuracy"] = "100.00"
df_copy.loc[df_copy["Time Series"] == "Chinatown", "Model Accuracy"] = "98.90"
df_copy.loc[df_copy["Time Series"] == "Coffee", "Model Accuracy"] = "53.33"
df_copy.loc[df_copy["Time Series"] == "Computers", "Model Accuracy"] = "53.33"
df_copy.loc[df_copy["Time Series"] == "CricketX", "Model Accuracy"] = "61.54"
# uni-tuning-df
df_copy.loc[df_copy["Time Series"] == "CricketY", "Model Accuracy"] = "60.51"
df_copy.loc[df_copy["Time Series"] == "CricketZ", "Model Accuracy"] = "62.56"
df_copy.loc[df_copy["Time Series"] == "Crop", "Model Accuracy"] = "76.78"

# Przygotowanie tabeli: wybieramy tylko istotne kolumny
final_cols = ["Time Series", "Model Accuracy", "Anchor", "LIME", "SHAP"]
# df_copy = df_copy[final_cols].copy()

# Filtrowanie na podstawie kolumny "type"
# Aby filtrowanie było niezależne od wielkości liter, używamy str.lower()
multivariate_table = df_copy[df_copy["type"].str.lower() == "multivariate"][final_cols].copy()
univariate_table   = df_copy[df_copy["type"].str.lower() == "univariate"][final_cols].copy()

# Konwersja tabel do formatu Markdown (metoda to_markdown() dostępna w nowszych wersjach pandas)
# multi_md = multivariate_table.to_markdown(index=False)
# uni_md = univariate_table.to_markdown(index=False)


def df_to_left_aligned_markdown(df: pd.DataFrame) -> str:
    """
    Zwraca łańcuch znaków zawierający tabelę w syntaksie Markdown
    z kolumnami wyrównanymi do lewej, z wykorzystaniem spacji.
    Kolumny będą poprawnie wyświetlane, jeśli używasz czcionki monospace.
    """
    # 1. Obliczamy potrzebne szerokości kolumn na podstawie zawartości i nagłówka
    col_widths = []
    for col in df.columns:
        # Maksymalna długość w kolumnie (włącznie z nagłówkiem)
        max_val_len = df[col].astype(str).apply(len).max()
        col_name_len = len(str(col))
        col_widths.append(max(max_val_len, col_name_len))

    # 2. Tworzymy wiersz nagłówka
    header_cells = []
    for i, col in enumerate(df.columns):
        col_str = str(col)
        header_cells.append(col_str.ljust(col_widths[i]))  # wyrównanie do lewej
    header_line = "| " + " | ".join(header_cells) + " |"

    # 3. Separator – aby wyrównać do lewej w Markdown,
    #    rozpoczynamy od dwukropka (":") i dalej dajemy kreski.
    #    Minimalny standard to ":-...", ale damy tyle kresek, co szerokość kolumny,
    #    aby w monospace wyglądało to równo.
    separator_cells = []
    for w in col_widths:
        # Zapewniamy minimum 3 kreski, tak by Markdown rozpoznał je jako tabelę
        if w < 3:
            w = 3
        # Dla wyrównania do lewej robimy `:---`
        # Natomiast aby wizualnie się zgadzało z ljust, używamy w kresek:
        cell = ":" + "-" * (w - 1)
        separator_cells.append(cell)
    separator_line = "| " + " | ".join(separator_cells) + " |"

    # 4. Wiersze z danymi
    data_lines = []
    for _, row in df.iterrows():
        row_cells = []
        for i, col in enumerate(df.columns):
            val_str = str(row[col])
            row_cells.append(val_str.ljust(col_widths[i]))
        row_line = "| " + " | ".join(row_cells) + " |"
        data_lines.append(row_line)

    # 5. Łączymy wszystko w jeden łańcuch
    markdown_table = "\n".join([header_line, separator_line] + data_lines)
    return markdown_table

# Generacja gotowych tabel w formacie Markdown ze stałymi odstępami:
multi_md = df_to_left_aligned_markdown(multivariate_table)
uni_md = df_to_left_aligned_markdown(univariate_table)

In [34]:
# Wypisanie gotowych tabel w blokach kodu, gotowych do wklejenia na Zenodo lub inny serwis
print("Multivariate Table:")
print("```markdown")
print(multi_md)
print("```")

Multivariate Table:
```markdown
| Time Series               | Model Accuracy | Anchor | LIME | SHAP |
| :------------------------ | :------------- | :----- | :--- | :--- |
| ArticularyWordRecognition | 96,53          | Yes    | Yes  | Yes  |
| AtrialFibrillation        | 37,50          | Yes    | Yes  | Yes  |
| BasicMotions              | 100,00         | Yes    | Yes  | Yes  |
| Cricket                   | 93,33          | Yes    | Yes  | Yes  |
| Epilepsy                  | 91,30          | Yes    | Yes  | Yes  |
| ERing                     | 98,67          | Yes    | Yes  | Yes  |
| EthanolConcentration      | 22,14          | Yes    | Yes  | Yes  |
| FaceDetection             | 50,25          | Yes    | No   | Yes  |
| FingerMovements           | 57,69          | Yes    | Yes  | Yes  |
| HandMovementDirection     | 23,73          | Yes    | Yes  | Yes  |
| Handwriting               | 58,40          | Yes    | Yes  | Yes  |
| Heartbeat                 | 72,82          | Yes    | Ye

In [35]:

print("\nUnivariate Table:")
print("```markdown")
print(uni_md)
print("```")


Univariate Table:
```markdown
| Time Series                    | Model Accuracy | Anchor | LIME | SHAP |
| :----------------------------- | :------------- | :----- | :--- | :--- |
| Adiac                          | 29.59          | No     | Yes  | Yes  |
| Beef                           | 53.33          | No     | Yes  | Yes  |
| BeetleFly                      | 80.00          | Yes    | Yes  | Yes  |
| BirdChicken                    | 50.00          | No     | Yes  | Yes  |
| BME                            | 86.67          | No     | Yes  | Yes  |
| CBF                            | 100.00         | No     | Yes  | Yes  |
| Chinatown                      | 98.90          | Yes    | Yes  | Yes  |
| Coffee                         | 53.33          | No     | Yes  | Yes  |
| Computers                      | 53.33          | No     | Yes  | Yes  |
| CricketX                       | 61.54          | Yes    | Yes  | Yes  |
| CricketY                       | 60.51          | Yes    | Yes  | Y